In [1]:
# ======================== 导入必要的库 ========================
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import itertools
import multiprocessing as mp

from copula_strategy_mpi import MPICopulaTradingRule
import copula_calculation as ccalc
from archimedean import (Gumbel, Clayton, Frank, Joe, N13, N14)
from elliptical import (StudentCopula, GaussianCopula)
from cjg_mix_copula import CJGMixCop  # 引入您提供的混合 Copula 类
from tqdm import tqdm
from scipy.stats import norm, rankdata
import matplotlib.pyplot as plt

In [2]:
# ======================== 读取数据并预处理 ========================
file_path = 'A_All_With_Interpolation.xlsx'  # Excel 文件路径
sheet_name = 'Sheet1'  # Excel 表格名称（根据实际情况修改）

# 读取 Excel 数据，并将 'Date' 列设置为索引
stock_prices = pd.read_excel(file_path, sheet_name=sheet_name)
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
stock_prices.set_index('Date', inplace=True)

# 确保所有股票代码列名格式一致（如果股票代码有前导零）
stock_prices.columns = stock_prices.columns.astype(str).str.zfill(6)

# 打印数据前几行，检查格式是否正确
print("股票数据前几行：")
print(stock_prices.head())

股票数据前几行：
            000012  000021  000025  000026  000039  000055  000060  000078   
Date                                                                         
2004-01-02   35.07   59.06    7.12   42.27   83.86   10.33   32.30   18.18  \
2004-01-05   35.91   61.77    7.04   40.95   86.28   10.46   33.92   18.03   
2004-01-06   37.13   66.05    7.17   41.44   86.75   10.08   35.76   17.97   
2004-01-07   37.02   67.48    7.12   40.95   85.97   10.29   35.60   18.30   
2004-01-08   38.66   71.77    7.30   41.83   85.97   10.75   35.38   18.69   

            000089  000096  ...  600861  600863  600867  600874  600881   
Date                        ...                                           
2004-01-02   24.16   11.40  ...   10.56   42.95   20.92    7.60   50.54  \
2004-01-05   25.06   11.95  ...   10.79   44.14   21.46    8.09   52.45   
2004-01-06   25.11   12.19  ...   10.86   43.66   21.68    8.13   51.97   
2004-01-07   25.63   12.33  ...   11.12   44.22   21.63    7.79   56.

In [3]:
# ======================== 设置形成期和交易期长度 ========================
formation_months = 12  # 形成期长度：12 个月
trading_months = 6     # 交易期长度：6 个月

# 设置起始日期和结束日期
start_date = pd.to_datetime('2004-01-01')  # 起始日期
end_date = stock_prices.index.max()        # 数据中的最后日期

# 初始化用于存储所有交易期每日回报的列表
all_daily_returns = []

In [3]:
# ======================== 开始时间循环 ========================
while start_date + pd.DateOffset(months=formation_months + trading_months) <= end_date:
    # 定义形成期和交易期的开始和结束日期
    formation_start_date = start_date
    formation_end_date = formation_start_date + pd.DateOffset(months=formation_months) - pd.Timedelta(days=1)

    trading_start_date = formation_end_date + pd.Timedelta(days=1)
    trading_end_date = trading_start_date + pd.DateOffset(months=trading_months) - pd.Timedelta(days=1)

    print(f"形成期：{formation_start_date.strftime('%Y-%m-%d')} 至 {formation_end_date.strftime('%Y-%m-%d')}")
    print(f"交易期：{trading_start_date.strftime('%Y-%m-%d')} 至 {trading_end_date.strftime('%Y-%m-%d')}")

    # 提取形成期和交易期的数据
    formation_data = stock_prices.loc[formation_start_date:formation_end_date]
    trading_data = stock_prices.loc[trading_start_date:trading_end_date]

    # ======================== 筛选股票对 ========================
    selected_stocks = stock_prices.columns.tolist()  # 您可以根据需要调整筛选标准

    # 生成股票对
    stock_pairs = list(itertools.combinations(selected_stocks, 2))

    # 定义最小数据长度要求（例如至少有 60 天的数据）
    minimum_length = 60

    # 初始化列表来存储每个股票对的 SSD 和 NZC 值
    pair_metrics = []

    # 获取形成期数据的列名
    formation_columns = formation_data.columns.tolist()

    # 遍历所有股票对，计算 SSD 和 NZC
    for stock1, stock2 in stock_pairs:
        # 获取形成期的价格数据
        price1 = formation_data[stock1]
        price2 = formation_data[stock2]

        # 对齐两个股票的价格数据，移除 NaN 值
        price1, price2 = price1.align(price2, join='inner')

        # 如果数据长度不足，跳过该股票对
        if len(price1) < minimum_length:
            continue

        # 计算价差
        spread = price1 - price2

        # 计算 SSD（价差的方差乘以数据点数量）
        deviations = spread - spread.mean()
        SSD = np.sum(deviations ** 2)

        # 计算 NZC（价差穿越零点的次数）
        spread_sign = np.sign(spread)
        zero_crossings = np.where(np.diff(spread_sign))[0]
        NZC = len(zero_crossings)

        # 将结果添加到列表中
        pair_metrics.append({
            'pair': (stock1, stock2),
            'SSD': SSD,
            'NZC': NZC
        })

    # 将结果转换为 DataFrame
    pair_metrics_df = pd.DataFrame(pair_metrics)

    # ================= 筛选股票对 =================

    # 计算 SSD 的第一个十分位（前 10%），即 SSD 最低的 10%
    SSD_first_decile = pair_metrics_df['SSD'].quantile(0.1)
    pairs_in_SSD_first_decile = pair_metrics_df[pair_metrics_df['SSD'] <= SSD_first_decile]

    # 计算 NZC 的第九个十分位（后 10%），即 NZC 最高的 10%
    NZC_ninth_decile = pair_metrics_df['NZC'].quantile(0.9)
    pairs_in_NZC_last_decile = pair_metrics_df[pair_metrics_df['NZC'] >= NZC_ninth_decile]

    # 取 SSD 和 NZC 筛选结果的交集
    SSD_pairs_set = set(pairs_in_SSD_first_decile['pair'])
    NZC_pairs_set = set(pairs_in_NZC_last_decile['pair'])
    intersect_pairs = SSD_pairs_set.intersection(NZC_pairs_set)

    # 从交集中获取股票对及其指标
    intersect_pairs_df = pair_metrics_df[pair_metrics_df['pair'].isin(intersect_pairs)]

    # 按照 SSD 从小到大排序
    intersect_pairs_df = intersect_pairs_df.sort_values('SSD')

    # 选择前 N 对股票对
    N = 60  # 您可以根据需要设置 N 的值
    top_N_pairs = intersect_pairs_df.head(N)

    # 显示选出的股票对
    print("选出的股票对：")
    print(top_N_pairs)

    # ================= 使用选出的前 20 对股票进行混合 Copula 拟合 =================

    # 初始化列表来存储每个股票对的 SIC 值和相关信息
    sic_results = []

    # 初始化计数器
    skipped_pairs = 0
    failed_pairs = 0

    # 遍历选出的前 20 对股票对，进行混合 Copula 拟合
    for index, row in tqdm(top_N_pairs.iterrows(), total=top_N_pairs.shape[0], desc="正在拟合股票对的混合 Copula"):
        stock1, stock2 = row['pair']

        # 获取形成期股票价格序列
        x = formation_data[stock1]
        y = formation_data[stock2]

        # 计算对数收益（Copula 策略中通常使用对数收益进行配对分析）
        x_returns = np.log(x).diff().dropna()
        y_returns = np.log(y).diff().dropna()

        # 拟合混合 Copula 模型，并计算拟合优度（SIC 值）
        try:
            # 将 x_returns 和 y_returns 组合成 DataFrame
            data = pd.DataFrame({'x': x_returns, 'y': y_returns})
        
            # 使用 ccalc 模块中的函数进行混合 Copula 拟合
            fit_result, copula_obj, cdf_x, cdf_y = ccalc.fit_copula_to_empirical_data(x=x_returns, y=y_returns, copula=CJGMixCop)
        
            # 获取 SIC 值
            sic_value = fit_result['SIC']
        
            # 将结果添加到列表中，并保存形成期的 Copula 模型和 CDF 函数
            sic_results.append({
                'pair': (stock1, stock2),
                'SIC': sic_value,
                'fitted_copula': copula_obj,   # 保存形成期拟合的 copula 对象
                'cdf_x': cdf_x,                # 保存股票1的 CDF
                'cdf_y': cdf_y                 # 保存股票2的 CDF
            })
        except Exception as e:
            # 记录拟合失败的股票对
            failed_pairs += 1
            print(f"股票对 {stock1}-{stock2} 拟合失败，错误：{e}")
            continue

    # 打印跳过的股票对和拟合失败的股票对数量
    print(f"跳过了 {skipped_pairs} 个股票对（缺失值或数据不足）。")
    print(f"拟合失败的股票对数量：{failed_pairs}。")

    # 如果 SIC 结果为空，输出提示信息
    if not sic_results:
        print("错误：没有成功生成 SIC 结果，跳过此周期。")
        start_date += pd.DateOffset(months=1)
        continue
    else:
        # 创建 DataFrame 并排序
        sic_df = pd.DataFrame(sic_results)
        sic_df_sorted = sic_df.sort_values('SIC')

        # 显示拟合成功的股票对及其 SIC 值
        print("拟合成功的股票对及其 SIC 值：")
        print(sic_df_sorted[['pair', 'SIC']])
    # 选择 SIC 最小的前 20 对股票对
    M = 20  # 您可以根据需要设置 M 的值
    top_M_pairs = sic_df_sorted.head(M)
    # ================= 执行交易策略 =================

    trading_results = []  # 用于记录每个股票对的交易结果

    # 遍历拟合成功的股票对
    for index, row in top_M_pairs.iterrows():
        stock1, stock2 = row['pair']
        print(f"执行股票对: {stock1}-{stock2} 的交易策略...")

        # ========== 数据准备 ==========
        # 获取形成期和交易期的数据
        formation_x = formation_data[stock1]
        formation_y = formation_data[stock2]
        trading_x = trading_data[stock1]
        trading_y = trading_data[stock2]

        # ========== 使用拟合的混合 Copula 模型 ==========
        copula_obj = row['fitted_copula']

        # ========== 创建 MPICopulaTradingRule 策略实例 ==========
        mpict_strategy = MPICopulaTradingRule(opening_triggers=(-0.6, 0.6), stop_loss_positions=(-2, 2))
        mpict_strategy.set_copula(copula_obj)
        mpict_strategy.set_cdf(cdf_x, cdf_y)

        # ========== 准备交易期的价格数据 ==========
        pair_prices = pd.DataFrame({
            stock1: trading_x,
            stock2: trading_y
        })

        # 计算收益率
        returns = mpict_strategy.to_returns(pair_prices)

        # 获取头寸和旗标
        positions, flags = mpict_strategy.get_positions_and_flags(returns)

        # 将头寸向前移动一位，以模拟实际交易
        positions = positions.shift(1)

        # 计算持有的单位数量，每个头寸的总价值为 1 RMB
        units_df = mpict_strategy.positions_to_units_dollar_neutral(pair_prices, positions, multiplier=1)

        # 计算每日持仓的盈亏（考虑到头寸总价值为 1 RMB）
        pnl = returns[stock1] * units_df[stock1] + returns[stock2] * units_df[stock2]

        # 在没有头寸的日子，盈亏为 0
        pnl[positions == 0] = 0

        # ========== 修改交易费用计算 ==========
        # 定义每只股票的持仓金额
        positions_stock1 = positions.apply(lambda x: 0.5 * x)
        positions_stock2 = positions.apply(lambda x: -0.5 * x)
    
        # 计算每只股票的持仓变化
        delta_positions_stock1 = positions_stock1.diff().fillna(0)
        delta_positions_stock2 = positions_stock2.diff().fillna(0)
    
        # 定义交易费用费率
        transaction_cost_rate = 0.0010
    
        # 计算每只股票的交易费用
        transaction_cost_stock1 = delta_positions_stock1.abs() * transaction_cost_rate
        transaction_cost_stock2 = delta_positions_stock2.abs() * transaction_cost_rate
    
        # 计算每日总交易费用
        transaction_cost = transaction_cost_stock1 + transaction_cost_stock2
    
        # 从每日盈亏中扣除交易费用
        pnl = pnl - transaction_cost

        # 将结果添加到 trading_results
        trading_results.append({
            'pair': (stock1, stock2),
            'positions': positions,
            'units_held': units_df,
            'daily_pnl': pnl
        })

    # ========== 汇总所有股票对的每日收益率 ==========
    combined_returns = pd.DataFrame(index=trading_data.index)

    # 将每个交易对的每日收益率汇总到 combined_returns
    for result in trading_results:
        pair = result['pair']
        daily_returns = result['daily_pnl']
        combined_returns[f"{pair[0]}-{pair[1]}"] = daily_returns

    # 计算所有股票对的总每日收益率（总收益率是所有股票对每日收益率的平均值）
    combined_returns['Total_Daily_Return'] = combined_returns.mean(axis=1)

    # 将当前交易期的每日回报添加到 all_daily_returns
    all_daily_returns.append(combined_returns['Total_Daily_Return'])

    # 将每日收益率保存为 Excel 文件或 CSV 文件（可选）
    output_filename = f'mixed_copula_daily_returns_{trading_start_date.strftime("%Y%m%d")}_{trading_end_date.strftime("%Y%m%d")}.xlsx'
    combined_returns.to_excel(output_filename, index=True)
    print(f"交易期每日收益率已保存为 '{output_filename}' 文件。")
    
    # 将形成期开始时间向前滚动一个月
    start_date += pd.DateOffset(months=1)

形成期：2004-01-01 至 2004-12-31
交易期：2005-01-01 至 2005-06-30
选出的股票对：
                   pair        SSD  NZC
33079  (600230, 600409)  11.187648   25
33063  (600230, 600336)  14.357967   27
36830  (600322, 600853)  16.222498   15
39085  (600382, 600409)  17.470966   20
35229  (600283, 600548)  18.986605   12
28213  (600116, 600278)  19.456103   35
37512  (600336, 600409)  20.112030   24
21766  (600007, 600382)  24.359355   18
33081  (600230, 600425)  25.248753   22
35343  (600285, 600391)  25.289844   20
37057  (600326, 600693)  25.791642   33
35326  (600285, 600329)  26.468707   14
17907  (000910, 600382)  27.230389   15
35196  (600283, 600368)  27.298602   14
23474  (600029, 600350)  27.519349   11
35833  (600300, 600693)  28.442516   17
31073  (600170, 600285)  28.860499   21
21431  (000993, 600853)  29.442343   33
14016  (000761, 600425)  30.149519   16
42319  (600548, 600789)  30.321199   12
35315  (600285, 600300)  30.625574   15
35752  (600300, 600329)  31.172604   13
20920  (000985, 

正在拟合股票对的混合 Copula:  23%|████████████▎                                        | 14/60 [01:07<02:40,  3.50s/it]C:\Users\Admin\Desktop\cjg_Mixed_Copula\cjg_mix_copula.py:236: RuntimeWarning: invalid value encountered in log
  log_likelihood_sum = np.sum(np.log(likelihood_list_mix))
正在拟合股票对的混合 Copula: 100%|█████████████████████████████████████████████████████| 60/60 [04:22<00:00,  4.37s/it]


跳过了 0 个股票对（缺失值或数据不足）。
拟合失败的股票对数量：0。
拟合成功的股票对及其 SIC 值：
                pair         SIC
16  (600170, 600285) -181.540965
11  (600285, 600329) -177.083014
36  (000915, 600693) -145.536488
10  (600326, 600693) -136.971316
23  (000151, 600693) -133.879109
22  (000985, 600562) -130.785374
9   (600285, 600391) -129.726709
56  (000151, 600391) -128.663767
30  (000705, 000957) -123.066095
8   (600230, 600425) -122.934834
29  (600329, 600391) -122.219333
20  (600285, 600300) -121.367716
40  (600326, 600562) -121.324418
31  (600368, 600382) -120.563621
38  (600336, 600425) -117.716066
33  (000915, 000985) -116.560152
15  (600300, 600693) -116.216409
50  (000937, 600348) -113.801433
48  (000701, 600809) -113.321436
47  (600336, 600368) -106.985023
13  (600283, 600368) -106.128623
12  (000910, 600382) -105.951206
3   (600382, 600409) -102.915820
52  (600326, 600509) -100.054936
21  (600300, 600329)  -99.492499
7   (600007, 600382)  -98.961365
45  (600192, 600853)  -97.945327
54  (600391, 600537)  

正在拟合股票对的混合 Copula:   0%|                                                              | 0/60 [00:03<?, ?it/s]

KeyboardInterrupt

